In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5
no_cuda = False
log_interval = 100

# Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Preprocess

In [4]:
train_paths = glob('dataset/mnist_png/training/*/*.png')[:1000]
test_paths = glob('dataset/mnist_png/testing/*/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

참고: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

In [5]:
class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):

        self.data_paths = data_paths
        self.transform = transform

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        path = self.data_paths[idx]
        image = Image.open(path).convert("L")
        label = int(path.split('\\')[-2])
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths, 
            transforms.Compose([
                transforms.RandomHorizontalFlip(), 
                transforms.ToTensor(), 
                transforms.Normalize(
                    mean=[0.406], 
                    std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths,
           transforms.Compose([
               transforms.ToTensor(), 
               transforms.Normalize(
                   mean=[0.406], 
                   std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=False
)

# Optimization

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# Training

In [8]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    
    
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    accuracy = 100. * correct / len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

Train Epoch: 1 [0/1000 (0%)]	Loss: 2.253301

Test set: Average loss: 0.6682, Accuracy: 980/1000 (98%)

Train Epoch: 2 [0/1000 (0%)]	Loss: 0.628125

Test set: Average loss: 0.1611, Accuracy: 980/1000 (98%)

Train Epoch: 3 [0/1000 (0%)]	Loss: 0.094323

Test set: Average loss: 0.1203, Accuracy: 980/1000 (98%)

Train Epoch: 4 [0/1000 (0%)]	Loss: 0.040491

Test set: Average loss: 0.1128, Accuracy: 980/1000 (98%)

Train Epoch: 5 [0/1000 (0%)]	Loss: 0.022716

Test set: Average loss: 0.1117, Accuracy: 980/1000 (98%)



# Save Model

공식 홈페이지에서는 바로 다음 단계에서 배울 전체 모델을 학습하는 것보다는 이 방법을 추천  
나중에 다른 모델에 적용하거나 수정하는 등 여러가지 이유로 용이하기 때문

https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [9]:
save_path = 'model_weight.pt'

In [12]:
torch.save(model.state_dict(), save_path)

In [10]:
model.state_dict().keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])

In [11]:
model.state_dict()['conv1.weight']

tensor([[[[ 0.1046, -0.0866, -0.0372,  0.0946, -0.1892],
          [ 0.1219, -0.0386,  0.1046,  0.0296, -0.0244],
          [ 0.0566,  0.0119,  0.0757, -0.0756, -0.0134],
          [-0.0185,  0.0291,  0.0004,  0.1769,  0.0639],
          [-0.0764, -0.1226, -0.0343, -0.0859, -0.0633]]],


        [[[ 0.0124,  0.1229,  0.1101, -0.1975,  0.1211],
          [ 0.0598,  0.1949,  0.1348, -0.1841, -0.1936],
          [-0.0927,  0.1809, -0.0301,  0.0855, -0.0956],
          [ 0.1994, -0.0810,  0.1531,  0.0031, -0.1068],
          [ 0.1043, -0.1045,  0.0607, -0.0570, -0.0225]]],


        [[[-0.1932, -0.0959,  0.1093, -0.0469,  0.2011],
          [ 0.1595, -0.0096, -0.1327,  0.1235,  0.0638],
          [-0.1299,  0.1299,  0.1220,  0.1782, -0.1117],
          [-0.0331, -0.0041,  0.0289, -0.1525, -0.1426],
          [ 0.1087, -0.0474,  0.0969,  0.0105,  0.0652]]],


        [[[ 0.0403,  0.0696,  0.0971, -0.1866,  0.1014],
          [-0.1477, -0.1575,  0.0089, -0.0350,  0.1187],
          [-0.1240,

In [13]:
model = Net().to(device)

In [14]:
weight_dict = torch.load(save_path)

In [15]:
weight_dict.keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])

In [16]:
model.load_state_dict(weight_dict)

<All keys matched successfully>

In [17]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

# Save Entire model

In [18]:
save_path = 'model.pt'

In [19]:
torch.save(model, save_path)

C:\Users\kcsgo\anaconda3\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## Load Model

In [20]:
model = torch.load(save_path)

In [21]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

# Save, Load and Resuming Training

In [22]:
checkpoint_path = 'checkpoint.pt'

In [23]:
torch.save({
    'epoch':epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}, checkpoint_path)

In [24]:
model = Net().to(device)

In [25]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [26]:
checkpoint = torch.load(checkpoint_path)

In [27]:
checkpoint

{'epoch': 5,
 'model_state_dict': OrderedDict([('conv1.weight',
               tensor([[[[ 0.1046, -0.0866, -0.0372,  0.0946, -0.1892],
                         [ 0.1219, -0.0386,  0.1046,  0.0296, -0.0244],
                         [ 0.0566,  0.0119,  0.0757, -0.0756, -0.0134],
                         [-0.0185,  0.0291,  0.0004,  0.1769,  0.0639],
                         [-0.0764, -0.1226, -0.0343, -0.0859, -0.0633]]],
               
               
                       [[[ 0.0124,  0.1229,  0.1101, -0.1975,  0.1211],
                         [ 0.0598,  0.1949,  0.1348, -0.1841, -0.1936],
                         [-0.0927,  0.1809, -0.0301,  0.0855, -0.0956],
                         [ 0.1994, -0.0810,  0.1531,  0.0031, -0.1068],
                         [ 0.1043, -0.1045,  0.0607, -0.0570, -0.0225]]],
               
               
                       [[[-0.1932, -0.0959,  0.1093, -0.0469,  0.2011],
                         [ 0.1595, -0.0096, -0.1327,  0.1235,  0.0638],
    

In [28]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [29]:
model

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [30]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [31]:
epoch

5

In [32]:
loss

tensor(0.0136, device='cuda:0', requires_grad=True)